In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
import os
from pyspark.sql.functions import udf, lead, unix_timestamp, monotonically_increasing_id, when, col, count, sum as spark_sum, row_number, expr, max as spark_max
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.types import StringType


os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'data-management-project-452400-74094d20e7ee.json'

GCS_CLICKSTREAM_PATH = 'gs://data-mgmt-bucket/parquet/clickstream_new'
input_path = "gs://data-mgmt-bucket/parquet/"

In [2]:
def spark_init():
    # Create SparkSession
    spark = (SparkSession.builder
        .appName("ETL")  # Set the application name
        .master("local[*]") #Optional - set master to local for local testing
        .config("spark.jars", "gcs-connector-hadoop3-latest.jar, spark-bigquery-with-dependencies_2.12-0.36.1.jar")
        .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") \
        .config("spark.hadoop.fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS") \
        .config("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
        .config("spark.hadoop.google.cloud.auth.service.account.json.keyfile", "data-management-project-452400-74094d20e7ee.json") \
        .getOrCreate())
    
    return spark

In [3]:
spark = spark_init()

25/03/14 01:20:00 WARN Utils: Your hostname, codespaces-c36745 resolves to a loopback address: 127.0.0.1; using 10.0.1.177 instead (on interface eth0)
25/03/14 01:20:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/03/14 01:20:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/14 01:20:02 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
clickstream_df = spark.read.format("parquet") \
    .option("header", "true") \
    .load(f'{input_path}/click_stream_new')

25/03/14 01:20:37 WARN FileSystem: Failed to initialize fileystem gs://data-mgmt-bucket/parquet/click_stream_new: java.io.FileNotFoundException: data-management-project-452400-74094d20e7ee.json (No such file or directory)
25/03/14 01:20:37 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: gs://data-mgmt-bucket/parquet//click_stream_new.
java.io.FileNotFoundException: data-management-project-452400-74094d20e7ee.json (No such file or directory)
	at java.base/java.io.FileInputStream.open0(Native Method)
	at java.base/java.io.FileInputStream.open(FileInputStream.java:213)
	at java.base/java.io.FileInputStream.<init>(FileInputStream.java:152)
	at java.base/java.io.FileInputStream.<init>(FileInputStream.java:106)
	at com.google.cloud.hadoop.repackaged.gcs.com.google.cloud.hadoop.util.CredentialFactory.getCredentialFromJsonKeyFile(CredentialFactory.java:297)
	at com.google.cloud.hadoop.repackaged.gcs.com.google.cloud.hadoop.util.Credent

Py4JJavaError: An error occurred while calling o43.load.
: java.io.FileNotFoundException: data-management-project-452400-74094d20e7ee.json (No such file or directory)
	at java.base/java.io.FileInputStream.open0(Native Method)
	at java.base/java.io.FileInputStream.open(FileInputStream.java:213)
	at java.base/java.io.FileInputStream.<init>(FileInputStream.java:152)
	at java.base/java.io.FileInputStream.<init>(FileInputStream.java:106)
	at com.google.cloud.hadoop.repackaged.gcs.com.google.cloud.hadoop.util.CredentialFactory.getCredentialFromJsonKeyFile(CredentialFactory.java:297)
	at com.google.cloud.hadoop.repackaged.gcs.com.google.cloud.hadoop.util.CredentialFactory.getCredential(CredentialFactory.java:414)
	at com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystemBase.getCredential(GoogleHadoopFileSystemBase.java:1479)
	at com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystemBase.createGcsFs(GoogleHadoopFileSystemBase.java:1638)
	at com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystemB

In [7]:
clickstream_df.printSchema()

root
 |-- session_id: string (nullable = true)
 |-- event_time: timestamp (nullable = true)
 |-- event_name: string (nullable = true)
 |-- product_id: double (nullable = true)



In [8]:
clickstream_df = clickstream_df.orderBy("session_id", "event_time")

In [9]:
clickstream_df.count()

25/03/12 23:59:17 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


12833602

In [10]:
windowSpec = Window.partitionBy("session_id").orderBy("event_time")

# Create the next_event_time column using the lead() function
df = clickstream_df.withColumn("next_event_time", lead("event_time").over(windowSpec))

# Compute dwell_time as the difference (in seconds) between next_event_time and event_time
df = df.withColumn("dwell_time", unix_timestamp("next_event_time") - unix_timestamp("event_time"))
df = df.filter(col("dwell_time") < 85000)

In [11]:
# 1. Assign a row number per session based on event_time order.
sessionWindow = Window.partitionBy("session_id").orderBy("event_time")
df_with_rn = df.withColumn("rn", row_number().over(sessionWindow))

# 2. For every row, compute a flag over the 3 preceding rows (rowsBetween(-3, -1))
# indicating if any of those rows is an ADD_TO_CART event.
w_preceding = Window.partitionBy("session_id").orderBy("rn").rowsBetween(-3, -1)
df_with_rn = df_with_rn.withColumn(
    "preceding_add_flag",
    spark_max(when(col("event_name") == "ADD_TO_CART", 1).otherwise(0)).over(w_preceding)
)

# 3. Extract valid ADD_TO_CART events.
# Only consider ADD_TO_CART events where none of the 3 preceding rows is ADD_TO_CART.
adds = df_with_rn.filter(col("event_name") == "ADD_TO_CART") \
         .filter(col("preceding_add_flag") == 0) \
         .select("session_id", "rn", col("product_id").alias("add_product_id"))

# 4. Identify candidate rows (irrespective of event_name) that have a null product_id.
candidates = df_with_rn.filter(col("product_id").isNull()) \
             .select("session_id", "rn")

# 5. Alias both DataFrames to avoid ambiguous columns.
cand_alias = candidates.alias("cand")
add_alias = adds.alias("add")

# Join candidates with valid ADD_TO_CART events in the same session where:
# candidate row number is between (add.rn - 3) and (add.rn - 1)
join_condition = (
    (col("cand.session_id") == col("add.session_id")) &
    (col("cand.rn") >= col("add.rn") - 3) &
    (col("cand.rn") < col("add.rn"))
)
joined = cand_alias.join(add_alias, join_condition, "inner") \
                   .select(
                        col("cand.session_id").alias("sess"),
                        col("cand.rn").alias("cand_rn"),
                        col("add.rn").alias("add_rn"),
                        col("add.add_product_id")
                   )

# 6. For each candidate (by session and candidate rn), select the ADD_TO_CART event with the smallest difference.
joined = joined.withColumn("diff", col("add_rn") - col("cand_rn"))
w_candidate = Window.partitionBy("sess", "cand_rn").orderBy("diff")
assignments = joined.withColumn("rn_assign", row_number().over(w_candidate)) \
                    .filter(col("rn_assign") == 1) \
                    .select(
                        col("sess").alias("session_id"),
                        col("cand_rn").alias("rn"),
                        col("add_product_id")
                    )

# 7. Join these assignments back to the original DataFrame.
df_updated = df_with_rn.join(assignments, on=["session_id", "rn"], how="left")

# 8. For candidate rows that got an assignment, update product_id.
df_updated = df_updated.withColumn(
    "final_product_id",
    when(col("add_product_id").isNotNull(), col("add_product_id")).otherwise(col("product_id"))
)

# 9. Clean up temporary columns.
result_df = df_updated.drop("rn", "preceding_add_flag", "add_product_id")

result_df.show(truncate=False)

25/03/12 23:59:53 WARN MemoryStore: Failed to reserve initial memory threshold of 1024.0 KiB for computing block broadcast_15 in memory.
25/03/12 23:59:53 WARN MemoryStore: Not enough space to cache broadcast_15 in memory! (computed 384.0 B so far)
25/03/12 23:59:53 WARN BlockManager: Persisting block broadcast_15 to disk instead.


+------------------------------------+--------------------------+-----------+----------+--------------------------+----------+----------------+
|session_id                          |event_time                |event_name |product_id|next_event_time           |dwell_time|final_product_id|
+------------------------------------+--------------------------+-----------+----------+--------------------------+----------+----------------+
|00001f29-3ede-4445-95f0-0178934e32ba|2019-05-17 11:51:52.276396|HOMEPAGE   |NULL      |2019-05-17 19:54:53.276396|28981     |33855.0         |
|00001f29-3ede-4445-95f0-0178934e32ba|2019-05-17 19:54:53.276396|ADD_TO_CART|33855.0   |2019-05-17 19:55:12.276396|19        |33855.0         |
|00001f29-3ede-4445-95f0-0178934e32ba|2019-05-19 04:07:20.276396|SCROLL     |NULL      |2019-05-19 20:17:04.276396|58184     |NULL            |
|00003eca-954b-4150-aee1-63fc62f395cf|2019-08-24 05:47:16.849738|HOMEPAGE   |NULL      |2019-08-24 05:59:38.849738|742       |43273.0   

In [12]:
result_df = result_df.na.drop(subset=['final_product_id'])
result_df = result_df.drop("product_id") \
    .withColumnRenamed("final_product_id", "product_id") \
    .withColumn("product_id", col("product_id").cast("int"))

In [13]:
result_df.orderBy('session_id', 'event_time').show()

25/03/13 00:00:59 WARN MemoryStore: Failed to reserve initial memory threshold of 1024.0 KiB for computing block broadcast_30 in memory.
25/03/13 00:00:59 WARN MemoryStore: Not enough space to cache broadcast_30 in memory! (computed 384.0 B so far)
25/03/13 00:00:59 WARN BlockManager: Persisting block broadcast_30 to disk instead.


+--------------------+--------------------+-----------+--------------------+----------+----------+
|          session_id|          event_time| event_name|     next_event_time|dwell_time|product_id|
+--------------------+--------------------+-----------+--------------------+----------+----------+
|00001f29-3ede-444...|2019-05-17 11:51:...|   HOMEPAGE|2019-05-17 19:54:...|     28981|     33855|
|00001f29-3ede-444...|2019-05-17 19:54:...|ADD_TO_CART|2019-05-17 19:55:...|        19|     33855|
|00003eca-954b-415...|2019-08-24 05:47:...|   HOMEPAGE|2019-08-24 05:59:...|       742|     43273|
|00003eca-954b-415...|2019-08-24 05:59:...| PROMO_PAGE|2019-08-24 06:00:...|        37|     43273|
|00003eca-954b-415...|2019-08-24 06:00:...|ADD_TO_CART|2019-08-24 06:26:...|      1557|     43273|
|0000425b-ddc8-445...|2019-03-04 20:42:...|ADD_TO_CART|2019-03-04 20:45:...|       198|     57690|
|00005746-17ec-449...|2022-03-08 23:17:...|   HOMEPAGE|2022-03-08 23:17:...|         3|      9980|
|00005746-

In [14]:
trans_df = spark.read.format("parquet") \
    .option("header", "true") \
    .load(f'{input_path}/transaction_new')\
    .select('created_at', 'session_id', 'payment_status', 'product_id', 'quantity')

In [15]:
click_agg = result_df.filter(col("event_name") == "CLICK") \
    .groupBy("product_id") \
    .agg(count("*").alias("click_count"))

item_detail_agg = result_df.filter(col("event_name") == "ITEM_DETAIL") \
    .groupBy("product_id") \
    .agg(count("*").alias("item_detail_count"))

add_to_cart_agg = result_df.filter(col("event_name") == "ADD_TO_CART") \
    .groupBy("product_id") \
    .agg(count("*").alias("add_to_cart_count"))

homepage_agg = result_df.filter(col("event_name") == "HOMEPAGE") \
    .groupBy("product_id") \
    .agg(count("*").alias("homepage_count"))

dwell_time_agg = result_df.groupBy("product_id") \
    .agg(spark_sum("dwell_time").alias("total_dwell_time"))

purchases_agg = trans_df.groupBy("product_id") \
    .agg(spark_sum("quantity").alias("purchases"))

In [16]:
popularity_df = click_agg.join(item_detail_agg, "product_id", "outer") \
    .join(add_to_cart_agg, "product_id", "outer") \
    .join(dwell_time_agg, "product_id", "outer") \
    .join(homepage_agg, "product_id", "outer") \
    .join(purchases_agg, "product_id", "outer")

In [17]:
popularity_df = popularity_df.na.fill(0)

In [18]:
product_df = spark.read.format("parquet") \
    .option("header", "true") \
    .load(f'{input_path}/product')

In [19]:
final_df = popularity_df.join(product_df, popularity_df.product_id == product_df.id, "inner")

In [20]:
final_df.write.format("bigquery") \
    .option("table", "data-management-project-452400.data_mgmt_project.click_event_agg_test") \
    .option("temporaryGcsBucket", "data-mgmt-bucket") \
    .mode("overwrite") \
    .save()

[244.955s][warning][gc,alloc] Executor task launch worker for task 1.0 in stage 88.0 (TID 613): Retried waiting for GCLocker too often allocating 4194306 words


25/03/13 00:03:07 WARN TaskMemoryManager: Failed to allocate a page (33554432 bytes), try again.
25/03/13 00:04:04 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/13 00:04:04 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/13 00:04:04 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/13 00:04:04 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/13 00:04:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/13 00:04:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/13 00:04:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/13 00:04:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBat

In [ ]:
clickstream_df[(clickstream_df['product_id'].notna())]['event_name'].value_counts()

event_name
ADD_TO_CART    113060
Name: count, dtype: int64

In [ ]:
clickstream_df.sort_values(['session_id', 'event_time'], inplace=True)
clickstream_df['next_event_time'] = clickstream_df.groupby('session_id')['event_time'].shift(-1)
clickstream_df['dwell_time'] = (clickstream_df['next_event_time'] - clickstream_df['event_time']).dt.total_seconds()
clickstream_df = clickstream_df[clickstream_df['dwell_time'] < 85000]

In [ ]:
clickstream_df['dwell_time'].describe()

count    908916.000000
mean       9933.780299
std       14295.619340
min           0.000000
25%         880.000000
50%        4345.000000
75%       12577.000000
max       84996.000000
Name: dwell_time, dtype: float64

In [ ]:
clickstream_df

,session_id,event_name,event_time,event_id,traffic_source,product_id,quantity,item_price,payment_status,search_keywords,promo_code,promo_amount,next_event_time,dwell_time
469318,00001f29-3ede-4445-95f0-0178934e32ba,HOMEPAGE,2019-05-17 11:51:52.276396+00:00,51793173-664c-4557-8a06-fc37587bea3d,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-05-17 19:54:53.276396+00:00,28981.0
469319,00001f29-3ede-4445-95f0-0178934e32ba,ADD_TO_CART,2019-05-17 19:54:53.276396+00:00,517b331c-4296-4772-a1a2-9bf7500a5570,MOBILE,33855.0,6.0,237967.0,NaN,NaN,NaN,NaN,2019-05-17 19:55:12.276396+00:00,19.0
469321,00001f29-3ede-4445-95f0-0178934e32ba,HOMEPAGE,2019-05-17 19:55:12.276396+00:00,2c0581d2-3185-4172-87ea-795ee26a4d0a,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-05-18 03:58:25.276396+00:00,28993.0
469322,00001f29-3ede-4445-95f0-0178934e32ba,CLICK,2019-05-18 03:58:25.276396+00:00,81982a1d-5562-44e8-8b1d-1c3366434c5e,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-05-18 12:02:44.276396+00:00,29059.0
469323,00001f29-3ede-4445-95f0-0178934e32ba,CLICK,2019-05-18 12:02:44.276396+00:00,6f7980bf-dce4-4b1e-9ffa-5c37499c4dc7,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-05-18 20:04:54.276396+00:00,28930.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
853329,fffe8be9-eef6-491c-b17c-07391bbf2387,ADD_TO_CART,2019-01-20 04:10:14.656243+00:00,7cd67d9c-e09e-448d-81f8-6e92d3e8ca59,MOBILE,29640.0,1.0,343988.0,NaN,NaN,NaN,NaN,2019-01-20 04:13:06.656243+00:00,172.0
853331,fffe8be9-eef6-491c-b17c-07391bbf2387,HOMEPAGE,2019-01-20 04:13:06.656243+00:00,7a2c177a-0fa8-4417-bc0c-a218380b2299,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-20 16:00:33.656243+00:00,42447.0
853332,fffe8be9-eef6-491c-b17c-07391bbf2387,ITEM_DETAIL,2019-01-20 16:00:33.656243+00:00,4e41d5fc-7dd4-4fc6-b183-7d72742e4980,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-21 03:41:16.656243+00:00,42043.0
853333,fffe8be9-eef6-491c-b17c-07391bbf2387,SCROLL,2019-01-21 03:41:16.656243+00:00,2174abb5-2020-49eb-af16-1429d3ec2b46,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-21 15:29:15.656243+00:00,42479.0


We can infer product_id for all events in a particular session

In [ ]:
clickstream_df['event_time'] = pd.to_datetime(clickstream_df['event_time'])
clickstream_df.sort_values(['session_id', 'event_time'], inplace=True)

def assign_inferred_product_id(session_df):
    # Ensure events are sorted by event_time.
    session_df = session_df.sort_values('event_time').reset_index(drop=True)
    # Initialize a new column to hold the inferred product id.
    session_df['inferred_product_id'] = None
    # Only events of type CLICK and ITEM_DETAIL are candidates.
    allowed_events = ['CLICK', 'ITEM_DETAIL']
    # Find the indices of all ADD_TO_CART events in this session.
    atc_indices = session_df.index[session_df['event_name'] == 'ADD_TO_CART'].tolist()
    # Set the start boundary for the candidate window.
    prev_bound = 0
    for atc_idx in atc_indices:
        # Get the product_id from the ADD_TO_CART event.
        product_id = session_df.loc[atc_idx, 'product_id']
        print(product_id)
        # Look for allowed events (CLICK or ITEM_DETAIL) between prev_bound and just before the ADD_TO_CART.
        candidates = session_df.loc[prev_bound:atc_idx-1]
        candidates = candidates[candidates['event_name'].isin(allowed_events)]
        if not candidates.empty:
            # Choose the last candidate (the most recent allowed event before the ADD_TO_CART).
            last_candidate_idx = candidates.index[-1]
            session_df.iloc[last_candidate_idx, 4] = product_id
            print('Inferred Successfully')
        # Update the boundary to be after the current ADD_TO_CART event.
        prev_bound = atc_idx + 1
    return session_df

# Apply the function to each session.
clickstream_df = clickstream_df.groupby('session_id', group_keys=False).apply(assign_inferred_product_id)

# Print the relevant columns.
clickstream_df[['session_id', 'event_name', 'event_time', 'product_id', 'inferred_product_id']]

33855.0
46727.0
4172.0
Inferred Successfully
7969.0
8626.0
Inferred Successfully
8958.0
39210.0
34363.0
45338.0
28137.0
27848.0
47003.0
10597.0
Inferred Successfully
51664.0
12955.0
51368.0
10885.0
18532.0
30636.0
32459.0
Inferred Successfully
55833.0
37263.0
24822.0
Inferred Successfully
53886.0
Inferred Successfully
18736.0
Inferred Successfully
29315.0
31040.0
21257.0
3456.0
37430.0
Inferred Successfully
14396.0
46589.0
28520.0
8323.0
Inferred Successfully
3451.0
11920.0
27999.0
41266.0
4787.0
Inferred Successfully
6061.0
Inferred Successfully
32316.0
54304.0
25595.0
44458.0
Inferred Successfully
3060.0
8968.0
38543.0
17435.0
59273.0
42207.0
19548.0
Inferred Successfully
17951.0
14576.0
23804.0
Inferred Successfully
20579.0
Inferred Successfully
57304.0
55037.0
18267.0
Inferred Successfully
6814.0
Inferred Successfully
20434.0
31185.0
39450.0
23795.0
49549.0
12403.0
25960.0
17099.0
5960.0
51426.0
52164.0
41404.0
19457.0
12000.0
28984.0
14659.0
32912.0
8903.0
19829.0
Inferred Success

C:\Users\utkar\AppData\Local\Temp\ipykernel_31884\343240909.py:32: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  clickstream_df = clickstream_df.groupby('session_id', group_keys=False).apply(assign_inferred_product_id)


,session_id,event_name,event_time,product_id,inferred_product_id
0,00001f29-3ede-4445-95f0-0178934e32ba,HOMEPAGE,2019-05-17 11:51:52.276396+00:00,NaN,None
1,00001f29-3ede-4445-95f0-0178934e32ba,ADD_TO_CART,2019-05-17 19:54:53.276396+00:00,33855.0,None
2,00001f29-3ede-4445-95f0-0178934e32ba,HOMEPAGE,2019-05-17 19:55:12.276396+00:00,NaN,None
3,00001f29-3ede-4445-95f0-0178934e32ba,CLICK,2019-05-18 03:58:25.276396+00:00,NaN,None
4,00001f29-3ede-4445-95f0-0178934e32ba,CLICK,2019-05-18 12:02:44.276396+00:00,NaN,None
...,...,...,...,...,...
1,fffe8be9-eef6-491c-b17c-07391bbf2387,ADD_TO_CART,2019-01-20 04:10:14.656243+00:00,29640.0,None
2,fffe8be9-eef6-491c-b17c-07391bbf2387,HOMEPAGE,2019-01-20 04:13:06.656243+00:00,NaN,None
3,fffe8be9-eef6-491c-b17c-07391bbf2387,ITEM_DETAIL,2019-01-20 16:00:33.656243+00:00,NaN,None
4,fffe8be9-eef6-491c-b17c-07391bbf2387,SCROLL,2019-01-21 03:41:16.656243+00:00,NaN,None


In [ ]:
clickstream_df['inferred_product_id'].notna().sum()

0

In [ ]:
# 1. Best-Selling Products Analysis

import pandas as pd

# Load transactions and product data
transactions = pd.read_csv("transactions.csv")
products = pd.read_csv("product.csv")

# Convert columns to correct data types
transactions['quantity'] = transactions['quantity'].astype(int)
transactions['item_price'] = transactions['item_price'].astype(float)

# Compute sales revenue per product
transactions['revenue'] = transactions['quantity'] * transactions['item_price']

# Aggregate data
best_sellers = (
    transactions.groupby("product_id")
    .agg(total_units_sold=("quantity", "sum"), total_revenue=("revenue", "sum"))
    .reset_index()
)

# Merge with product details
best_sellers = best_sellers.merge(products, left_on="product_id", right_on="id")

# Get top 10 best-selling products
top_products = best_sellers.sort_values(by="total_revenue", ascending=False).head(10)

# Display results
# tools.display_dataframe_to_user(name="Top Selling Products", dataframe=top_products)

ParserError: Error tokenizing data. C error: Expected 10 fields in line 6044, saw 11


: 

In [ ]:
import pandas as pd

# Convert event_time to datetime if not already done.
clickstream_df['event_time'] = pd.to_datetime(clickstream_df['event_time'])

# Sort by session_id and event_time to ensure correct order.
clickstream_df.sort_values(['session_id', 'event_time'], inplace=True)

def assign_product_id_to_session(group):
    # Sort events within the session.
    group = group.sort_values('event_time').reset_index(drop=True)
    # Initialize inferred_product_id column to None.
    group['inferred_product_id'] = None
    # Only consider these event types when looking backward.
    allowed_events = ['CLICK', 'ITEM_DETAIL']
    
    # Get indices of all ADD_TO_CART events in this session.
    atc_indices = group.index[group['event_name'] == 'ADD_TO_CART'].tolist()
    
    # For each ADD_TO_CART event, find the last allowed event preceding it.
    for atc_idx in atc_indices:
        # Product id from the current ADD_TO_CART event.
        product_id = group.loc[atc_idx, 'product_id']
        # Identify candidate events (with allowed event types) that come before this ADD_TO_CART.
        candidate_indices = group.loc[:atc_idx-1].index[group.loc[:atc_idx-1, 'event_name'].isin(allowed_events)]
        if len(candidate_indices) > 0:
            # Use the last candidate (the one with the highest index, i.e. latest before ADD_TO_CART).
            last_allowed_index = candidate_indices[-1]
            group.loc[last_allowed_index, 'inferred_product_id'] = product_id
    return group

# Apply the function to each session.
clickstream_df = clickstream_df.groupby('session_id', group_keys=False).apply(assign_product_id_to_session)

# Now, filter to engagement events of interest (CLICK and ITEM_DETAIL) that have an inferred product id.
engagement_events = clickstream_df[
    clickstream_df['event_name'].isin(['CLICK', 'ITEM_DETAIL']) & 
    clickstream_df['inferred_product_id'].notnull()
]

# Aggregate engagement metrics by the inferred product_id.
engagement_metrics = engagement_events.groupby('inferred_product_id').agg(
    event_count=('event_name', 'count')
).reset_index().rename(columns={'inferred_product_id': 'product_id'})

engagement_metrics.head()

C:\Users\utkar\AppData\Local\Temp\ipykernel_31884\2858203197.py:33: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  clickstream_df = clickstream_df.groupby('session_id', group_keys=False).apply(assign_product_id_to_session)


,product_id,event_count
0,1525.0,1
1,1526.0,1
2,1528.0,1
3,1536.0,1
4,1537.0,2


,session_id,event_name,event_time,event_id,traffic_source,product_id,quantity,item_price,payment_status,search_keywords,promo_code,promo_amount,inferred_product_id
1,00001f29-3ede-4445-95f0-0178934e32ba,ADD_TO_CART,2019-05-17 19:54:53.276396+00:00,517b331c-4296-4772-a1a2-9bf7500a5570,MOBILE,33855.0,6.0,237967.0,NaN,NaN,NaN,NaN,None
3,00001f29-3ede-4445-95f0-0178934e32ba,CLICK,2019-05-18 03:58:25.276396+00:00,81982a1d-5562-44e8-8b1d-1c3366434c5e,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
4,00001f29-3ede-4445-95f0-0178934e32ba,CLICK,2019-05-18 12:02:44.276396+00:00,6f7980bf-dce4-4b1e-9ffa-5c37499c4dc7,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
5,00001f29-3ede-4445-95f0-0178934e32ba,ITEM_DETAIL,2019-05-18 20:04:54.276396+00:00,02d7e74f-3f96-41ab-9a5d-26c2ee392f06,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
7,00001f29-3ede-4445-95f0-0178934e32ba,BOOKING,2019-05-19 20:17:04.276396+00:00,e0d13c13-b9ca-4ea7-9560-78a47f45a20b,MOBILE,NaN,NaN,NaN,Success,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,fffdbe04-e042-4859-9336-7cf4eda9a1f3,CLICK,2022-03-27 20:16:22.669060+00:00,7f56122b-ddda-4f40-90ce-0958923a7eb2,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
3,fffdbe04-e042-4859-9336-7cf4eda9a1f3,BOOKING,2022-03-30 03:24:46.669060+00:00,b5e42b2b-8885-43b7-960b-ee4895e4521b,MOBILE,NaN,NaN,NaN,Success,NaN,NaN,NaN,None
1,fffe8be9-eef6-491c-b17c-07391bbf2387,ADD_TO_CART,2019-01-20 04:10:14.656243+00:00,7cd67d9c-e09e-448d-81f8-6e92d3e8ca59,MOBILE,29640.0,1.0,343988.0,NaN,NaN,NaN,NaN,None
3,fffe8be9-eef6-491c-b17c-07391bbf2387,ITEM_DETAIL,2019-01-20 16:00:33.656243+00:00,4e41d5fc-7dd4-4fc6-b183-7d72742e4980,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


In [ ]:
engagement_events

,session_id,event_name,event_time,event_id,traffic_source,product_id,quantity,item_price,payment_status,search_keywords,promo_code,promo_amount,inferred_product_id
1,000288dc-e7cf-44b9-9f2d-270015fd83c6,ITEM_DETAIL,2021-11-20 00:38:17.288196+00:00,e89fbcee-3171-4639-9a1d-3156a72a9bc2,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4172.0
1,0004012d-a905-4411-9ed4-2f80778feafe,ITEM_DETAIL,2020-12-27 09:28:24.397792+00:00,5e93c10d-9ce0-4618-b6ef-a51042b0c1db,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8958.0
7,0004a8e1-2c96-4f35-a957-560473dd78e5,CLICK,2020-11-11 04:46:44.690972+00:00,bf3cbf56-f660-4532-97bb-f7bed53c7ac6,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12955.0
2,000bb4a4-219c-4eab-a0ea-7a5c2bd5d8e8,CLICK,2019-06-17 09:05:33.624154+00:00,a125fa73-16f5-428a-bc89-21896c0622fe,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37263.0
8,000bb4a4-219c-4eab-a0ea-7a5c2bd5d8e8,CLICK,2019-06-17 15:51:57.624154+00:00,c3437ed6-a7cf-4642-8ddb-b571091b5741,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24822.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,ffebe1a6-fe25-4272-87cb-ac75acd9c47e,CLICK,2020-05-29 15:20:09.676325+00:00,de33b854-677d-4121-be60-109ae9a2cb66,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34227.0
8,fff4870b-f0e8-4563-ad10-1e5f6ef8c13f,ITEM_DETAIL,2019-02-16 22:05:14.162212+00:00,d98fc027-0824-4a9a-bcb9-ed6a5184190b,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54128.0
1,fff5f45a-6aee-4f6e-a6c0-3318b3687ee8,CLICK,2021-04-21 06:35:39.351064+00:00,516cbad8-ea39-40ab-a89e-2eec22ad507d,WEB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15545.0
2,fff9768b-2039-46be-8bcd-abe47d181cdb,CLICK,2019-06-03 12:52:19.187897+00:00,046f1484-a3df-441b-978a-999520c93b8f,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12882.0


In [ ]:
clickstream.printSchema()

NameError: name 'clickstream' is not defined

In [ ]:
product = pd.read_csv('product.csv')

ParserError: Error tokenizing data. C error: Expected 10 fields in line 6044, saw 11


## PySpark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize SparkSession
spark = SparkSession.builder.appName("InferredProductID").getOrCreate()

# Load your clickstream data into a DataFrame, for example from a CSV.
# (Assuming the CSV has a header and columns as in your sample.)
df = spark.read.option("header", "true").csv("click_stream_new.csv")

# Convert event_time to timestamp (adjust format if needed)
df = df.withColumn("event_time", col("event_time").cast("timestamp"))

# Create or replace a temporary view to use SQL.
df.createOrReplaceTempView("clickstream")

# Spark SQL query to assign inferred product_id.
query = """
WITH atc_raw AS (
  SELECT 
    session_id, 
    event_time AS atc_time, 
    product_id,
    LAG(event_time) OVER (PARTITION BY session_id ORDER BY event_time) AS prev_atc_time
  FROM clickstream
  WHERE event_name = 'ADD_TO_CART'
),
allowed AS (
  SELECT 
    session_id, 
    event_time AS allowed_time, 
    event_name
  FROM clickstream
  WHERE event_name IN ('CLICK', 'ITEM_DETAIL', 'ADD_TO_CART')
),
joined AS (
  SELECT 
    a.session_id,
    a.allowed_time,
    a.event_name,
    t.atc_time,
    t.product_id,
    ROW_NUMBER() OVER (
      PARTITION BY t.session_id, t.atc_time 
      ORDER BY a.allowed_time DESC
    ) AS rn
  FROM allowed a
  JOIN atc_raw t 
    ON a.session_id = t.session_id 
   AND a.allowed_time < t.atc_time
   AND (t.prev_atc_time IS NULL OR a.allowed_time > t.prev_atc_time)
)
SELECT 
  session_id,
  allowed_time AS event_time,
  event_name,
  product_id AS inferred_product_id
FROM joined
WHERE rn = 1
"""

# Run the query.
result_df = spark.sql(query)

# Show the results.
result_df.show()


+--------------------+--------------------+-----------+-------------------+
|          session_id|          event_time| event_name|inferred_product_id|
+--------------------+--------------------+-----------+-------------------+
|0004a700-7db4-412...|2021-08-29 11:21:...|      CLICK|            33771.0|
|00080cfb-06ab-4db...|2021-02-17 19:51:...|      CLICK|            48164.0|
|000d9eb3-ac60-4b7...|2019-11-13 23:28:...|      CLICK|             8323.0|
|000ddfd6-8d0b-41b...|2021-03-23 21:47:...|      CLICK|            51701.0|
|000ddfd6-8d0b-41b...|2021-03-24 09:53:...|      CLICK|            48922.0|
|0011f585-d0e9-401...|2022-01-03 16:15:...|      CLICK|             5600.0|
|00153f8c-623b-498...|2021-06-02 23:26:...|      CLICK|            13814.0|
|00153f8c-623b-498...|2021-06-19 00:37:...|      CLICK|            44421.0|
|001bd09f-a9db-4bb...|2020-06-10 06:45:...|ITEM_DETAIL|            18267.0|
|001bd09f-a9db-4bb...|2020-06-10 10:48:...|      CLICK|             6814.0|
|001c45cb-7e

In [ ]:
# print the length of the spark datraframe
print(result_df.count())

607434


In [9]:
import os
import pandas as pd
from google.cloud import bigquery
import json

from google.cloud import storage


PROJECT_ID = "data-management-project-452400"
BUCKET_NAME = "data-mgmt-bucket"

# BigQuery details
BQ_DATASET_NAME = "data_mgmt_project"
BQ_PRODUCT_POPULARITY_TABLE_NAME = "top10_product_popularity"
BQ_BRAND_POPULARITY_TABLE_NAME = "top10_brand_popularity"
BQ_sub_category_POPULARITY_TABLE_NAME = "top10_sub_category_popularity"
BQ_gender_season_POPULARITY_TABLE_NAME = "top_10_products_by_season_gender"


POP_WEIGHTS_PATH = 'gs://data-mgmt-bucket/feature_importance.json'
# Set up Google Cloud authentication (Ensure you've set up your service account JSON)
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'authkey.json'

def load_to_bq(client, df, table_path):
    job = client.load_table_from_dataframe(df, table_path, job_config=bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE"))
    return job.result()

# ============================
# 1️⃣  CONNECT TO BIGQUERY
# ============================

# Initialize BigQuery Client
client = bigquery.Client()

# Define your BigQuery table
bq_table = "data-management-project-452400.data_mgmt_project.brands_data"

# Query BigQuery to get the data
query = f"""
SELECT 
    *
FROM `{bq_table}`
"""

# Load Data into Pandas DataFrame
df = client.query(query).to_dataframe()


# Read in the weights for product popularity computation from GCS
bucket_name = BUCKET_NAME # Replace with actual bucket name
file_name = "feature_importance.json"  # JSON file stored in GCS

# Initialize GCS client
client = storage.Client()

# Get the GCS bucket and file
bucket = client.bucket(bucket_name)
blob = bucket.blob(file_name)

# Download JSON file as text and load it as a dictionary
json_data = json.loads(blob.download_as_text())

# Convert to Pandas Series instead of DataFrame
weights = pd.Series(json_data)

df["popularity_score"] = (df["click_count"] * weights["click_count"] +
    df["item_detail_count"] * weights["item_detail_count"] +
    df["add_to_cart_count"] * weights["add_to_cart_count"] +
    df["total_dwell_time"] * weights["total_dwell_time"] +
    df["homepage_count"] * weights["homepage_count"]+
    df["purchases"] * weights["purchases"])/6

df["popularity_score"] = (df["popularity_score"] - df["popularity_score"].min()) / (df["popularity_score"].max() - df["popularity_score"].min())


## Most Popular Products

In [10]:
########## Most Popular Products ##########
# Get top 10 most popular products
df_popular_products = df.sort_values(by="popularity_score", ascending=False).head(10)

In [11]:
import duckdb
duckdb.sql("SELECT 'Hello, DuckDB!'")

┌──────────────────┐
│ 'Hello, DuckDB!' │
│     varchar      │
├──────────────────┤
│ Hello, DuckDB!   │
└──────────────────┘

In [12]:
duckdb_file = "/tmp/duckdb_database.db"  # Local path
con = duckdb.connect(duckdb_file)
con.execute("CREATE TABLE IF NOT EXISTS top10_popular_products AS SELECT * FROM df_popular_products")

In [13]:
###### Loading to BigQuery
try:
    client = bigquery.Client()
    BQ_PATH = f"{PROJECT_ID}.{BQ_DATASET_NAME}.{BQ_PRODUCT_POPULARITY_TABLE_NAME}"
    job = client.load_table_from_dataframe(df_popular_products, BQ_PATH, job_config=bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE"))
    print(f'Succussfully Upload table: {BQ_PRODUCT_POPULARITY_TABLE_NAME}')
    
except Exception as e:
    print(e)


Succussfully Upload table: top10_product_popularity


## Most Popular Brands

In [14]:
########## Most Popular Brands ##########
brand_popularity = df.groupby("brands", as_index=False)["popularity_score"].sum()
brand_popularity["popularity_score"] = (brand_popularity["popularity_score"] - brand_popularity["popularity_score"].min()) / (brand_popularity["popularity_score"].max() - brand_popularity["popularity_score"].min())
brand_popularity = brand_popularity.sort_values(by="popularity_score", ascending=False).head(10)

In [15]:
con.execute("CREATE TABLE IF NOT EXISTS top10_popular_brands AS SELECT * FROM brand_popularity")

In [16]:
try:
    client = bigquery.Client()
    BQ_PATH = f"{PROJECT_ID}.{BQ_DATASET_NAME}.{BQ_BRAND_POPULARITY_TABLE_NAME}"
    job = client.load_table_from_dataframe(brand_popularity, BQ_PATH, job_config=bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE"))
    print (f'Succussfully Upload table: {BQ_BRAND_POPULARITY_TABLE_NAME}')

except Exception as e:
    print("Error loading to BigQuery ", e)

Succussfully Upload table: top10_brand_popularity


## Most Popular Sub Categories

In [17]:
########## Most Popular Sub Categories ##########
subcategory_popularity = df.groupby("subCategory", as_index=False)["popularity_score"].sum()
subcategory_popularity["popularity_score"] = (subcategory_popularity["popularity_score"] - subcategory_popularity["popularity_score"].min()) / (subcategory_popularity["popularity_score"].max() - subcategory_popularity["popularity_score"].min())
subcategory_popularity = subcategory_popularity.sort_values(by="popularity_score", ascending=False).head(10)


In [18]:
con.execute("CREATE TABLE IF NOT EXISTS top10_popular_subcategories AS SELECT * FROM subcategory_popularity")

In [19]:
try:
    client = bigquery.Client()
    BQ_PATH = f"{PROJECT_ID}.{BQ_DATASET_NAME}.{BQ_sub_category_POPULARITY_TABLE_NAME}"
    job = client.load_table_from_dataframe(subcategory_popularity, BQ_PATH, job_config=bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE"))
    print (f'\nSuccussfully Upload table: {BQ_sub_category_POPULARITY_TABLE_NAME}\n')
except Exception as e:
    print(e)


Succussfully Upload table: top10_sub_category_popularity



## Gender and Season

In [20]:

########## Most Popular Products by Season and Gender ##########
# ============================
# 1️⃣  CONNECT TO BIGQUERY
# ============================

# Initialize BigQuery Client
client = bigquery.Client()

# Define your BigQuery table
bq_table = "data-management-project-452400.data_mgmt_project.click_event_agg"

# Query BigQuery to get the data
query = f"""
SELECT 
    product_id, productDisplayName, gender
FROM `{bq_table}`
"""

# Load Data into Pandas DataFrame
df1 = client.query(query).to_dataframe()
df_final = pd.merge(df, df1, left_on="product_id", right_on="product_id", how="outer")

top_10_products_by_season_gender = (
    df_final.groupby(["season", "gender"])
    .apply(lambda x: x.nlargest(10, "popularity_score"))
    .reset_index(drop=True)
)

/tmp/ipykernel_10974/1059062906.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.nlargest(10, "popularity_score"))


In [21]:
con.execute("CREATE TABLE IF NOT EXISTS top_10_products_by_season_gender AS SELECT * FROM top_10_products_by_season_gender")

In [ ]:
try:
    client = bigquery.Client()
    BQ_PATH = f"{PROJECT_ID}.{BQ_DATASET_NAME}.{BQ_gender_season_POPULARITY_TABLE_NAME}"
    job = client.load_table_from_dataframe(top_10_products_by_season_gender, BQ_PATH, job_config=bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE"))
    print (f'Succussfully Upload table: {BQ_gender_season_POPULARITY_TABLE_NAME}')
except Exception as e:
    print(e)

Succussfully Upload table: top_10_products_by_season_gender


In [30]:
from google.cloud import storage

def upload_to_gcs(bucket_name, source_file_path, destination_blob_name):
    """Uploads a file to Google Cloud Storage."""
    
    # Initialize a GCS client
    client = storage.Client()

    # Get the bucket
    bucket = client.bucket(bucket_name)

    # Create a blob (file in GCS)
    blob = bucket.blob(destination_blob_name)

    # Upload the file
    blob.upload_from_filename(source_file_path)

    print(f"✅ File {source_file_path} uploaded to gs://{bucket_name}/{destination_blob_name}")

# Example usage:
bucket_name = "data-mgmt-bucket"  # Your GCS bucket name
source_file_path = "/tmp/duckdb_database.db"  # Path to your DuckDB file
destination_blob_name = "duckdb/duckdb_database.db"  # Target location in GCS

upload_to_gcs(bucket_name, source_file_path, destination_blob_name)


✅ File /tmp/duckdb_database.db uploaded to gs://data-mgmt-bucket/duckdb/duckdb_database.db
